<a href="https://colab.research.google.com/github/pritiyadav888/Machine-Learning-Projects/blob/main/google_vantilaor_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import gc

In [8]:
# # load the dataset
# train = loadtxt('../input/ventilator-pressure-prediction/train.csv', delimiter=',')
# test = loadtxt('../input/ventilator-pressure-prediction/test.csv', delimiter=',')
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/test.csv')
# sample = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

In [9]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


**Subsample- 20% data**

In [10]:
X = train.copy()
y = X.pop('pressure')

In [11]:
X = np.array(X)
X

array([[      1.   ,       1.   ,      20.   , ...,       0.   ,
              0.083,       0.   ],
       [      2.   ,       1.   ,      20.   , ...,       0.034,
             18.383,       0.   ],
       [      3.   ,       1.   ,      20.   , ...,       0.068,
             22.509,       0.   ],
       ...,
       [6035998.   ,  125749.   ,      50.   , ...,       2.571,
              1.559,       1.   ],
       [6035999.   ,  125749.   ,      50.   , ...,       2.605,
              1.273,       1.   ],
       [6036000.   ,  125749.   ,      50.   , ...,       2.638,
              1.483,       1.   ]])

In [12]:
model = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

model.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.Adam())

In [13]:
# from keras import callbacks
earlystopping = tf.keras.callbacks.EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

In [15]:
model.fit(X, y, batch_size = 32, 
                    epochs = 10, 
                    callbacks =[earlystopping])

Epoch 1/10
188625/188625 [==============================] - 233s 1ms/step - loss: 4164417.2500
Epoch 2/10
188625/188625 [==============================] - 227s 1ms/step - loss: 1357250.2500
Epoch 3/10
188625/188625 [==============================] - 225s 1ms/step - loss: 398681.0312
Epoch 4/10
188625/188625 [==============================] - 231s 1ms/step - loss: 120456.3984
Epoch 5/10
188625/188625 [==============================] - 225s 1ms/step - loss: 28287.0312
Epoch 6/10
188625/188625 [==============================] - 223s 1ms/step - loss: 8496.7354
Epoch 7/10
188625/188625 [==============================] - 226s 1ms/step - loss: 2932.7178
Epoch 8/10
188625/188625 [==============================] - 225s 1ms/step - loss: 3087.8132
Epoch 9/10
188625/188625 [==============================] - 224s 1ms/step - loss: 2690.9031
Epoch 10/10
188625/188625 [==============================] - 223s 1ms/step - loss: 3536.6724


In [16]:
gc.collect()

824

In [17]:
# make class predictions with the model
predictions = model.predict_classes(X)
# summarize the first 5 cases
for i in range(5):
	print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))


AttributeError: ignored

In [ ]:
gc.collect()

In [18]:
predicted = model.predict(test)
predicted

array([[ 10.508],
       [ 12.167],
       [ 13.742],
       ...,
       [-36.908],
       [-36.934],
       [-36.959]], dtype=float32)

In [19]:
test_result = test
test_result['pressure'] = predicted

In [20]:
submit_result = test_result[['id','pressure']]
submit_result

,id,pressure
0,1,10.508467
1,2,12.167415
2,3,13.741582
3,4,15.190825
4,5,16.306648
...,...,...
4023995,4023996,-36.858738
4023996,4023997,-36.883602
4023997,4023998,-36.908489
4023998,4023999,-36.933605


In [21]:
submit_result.to_csv('/content/drive/MyDrive/Colab Notebooks/Predictive_Data/vantilator_google/submission.csv', index=False)